In [4]:
# import numpy as np
import math
import numpy as np
import torch # Used only for matrix multiplication
from torch.autograd import Variable
import pandas as pd
import gc
import tensorflow as tf # Used only to acquire MNIST data
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
X = []
device = torch.device("cpu")
# Probably an input of images (eg MNIST or something)
num_layers = 4
dtype = torch.float
torch.seed = 9
np.random.seed(9)

# Convolutional Neural Network written from scratch for basic classification tasks (eg MNIST / Cats and Dogs)
# Can be used as a discriminator for a GAN as well

# CNN_Engine is sort of complete. Now you have to clean up the code and make it bug free 
# Also, further optimisation to improve speed would be good. Use dictionaries wherever possible

class CNN_Engine:
    def __init__(self,num_layers,X,y,loss_type,num_epochs,bs,stride,dropout):
        self.num_layers = num_layers
        self.X = X
        self.y = y
        self.loss_type = loss_type      
        self.num_epochs = num_epochs
        self.bs = bs
        self.padding = padding
        self.stride = stride
        self.dropout = dropout
        
    def cost_function(self,y_pred,y): # Assuming cross entropy loss
        sumx = 0
        for k in self.y:
            sumx += -k*math.log(y_pred)
        return sumx

    def Relu(self,x):
        if self.x > 0:
            return self.x
        else:
            return 0
    
    def MaxPool2d(self,X,kernel_size,stride):
        return

    def BatchNorm(self,X):
        return (X-torch.mean(X)) / torch.std(X)
    
    def im2col2d(X,kernel,stride): # Assuming X and the kernel are 2D inputs
        # We transform the input matrix X into a matrix Ω(x) such that the columns correspond to the 
        # set of elements that will be multiplied by the kernel in each sliding convolution operation
        # Given x -> Ω(x), conv2d(x,kernel) = Ω(x)' * kernel where ' denotes a transpose operation

        X = X.detach().numpy()
        X = np.array(X)
        kernel_shape = np.array(list(np.shape(kernel))).T
        img_shape = np.array(list(np.shape(X))).T
        a = np.empty((kernel_shape[0]*kernel_shape[1]*kernel_shape[2]))
        output_matrix = np.reshape(a,(kernel_shape[0]*kernel_shape[1]*kernel_shape[2],1))
        for k in range(0,img_shape[2],stride):
             for j in range(0,img_shape[1],stride):
                for i in range(0,img_shape[0],stride):
                        try:
                            vals = X[i:i+kernel_shape[0],j:j+kernel_shape[1],k:k+kernel_shape[2]]
                            vals = np.reshape(vals,(int(np.shape(vals)[2]*np.shape(vals)[1]*np.shape(vals)[0]),1))
                            output_matrix = np.hstack((output_matrix,vals))
                        except:
                            continue
        
        output_matrix = np.delete(output_matrix,0,1)
        output_matrix = torch.FloatTensor(output_matrix)
        return torch.nn.Parameter(output_matrix)

    def conv2D(self,channels,X,stride,kernel_shape,padding = False): # filters = dimensionality of output space
            # If padding is enabled, we pad the input with zeros such that the input size 
            # remains the same if weights with stride 1 are applied to the input
            kernel = np.random.uniform(-0.1,0.1,size = (kernel_shape[0],kernel_shape[1],kernel_shape[2])) # Our input
            kernel = torch.FloatTensor(kernel)
            kernel.requires_grad = True
            X = X.detach().numpy()
            if padding:
                if kernel_shape[1] % 2 == 0 and kernel_shape[2] % 2 == 0:
                    X = np.pad(X,((0,0),(math.floor(kernel_shape[1]/2)-1,math.floor(kernel_shape[1]/2)),(math.floor(kernel_shape[2]/2),math.floor(kernel_shape[2]/2)-1)), 'symmetric')
                elif kernel_shape[1] % 2 != 0 and kernel_shape[2] % 2 == 0:
                    X = np.pad(X,((0,0),(math.floor(kernel_shape[1]/2),math.floor(kernel_shape[1]/2)),(math.floor(kernel_shape[2]/2),math.floor(kernel_shape[2]/2)-1)), 'symmetric')
                elif kernel_shape[1] % 2 == 0 and kernel_shape[2] % 2 != 0:
                    X = np.pad(X,((0,0),(math.floor(kernel_shape[1]/2)-1,math.floor(kernel_shape[1]/2)),(math.floor(kernel_shape[2]/2),math.floor(kernel_shape[2]/2))), 'symmetric')
                else:
                    X = np.pad(X,((0,0),(math.floor(kernel_shape[1]/2),math.floor(kernel_shape[1]/2)),(math.floor(kernel_shape[2]/2),math.floor(kernel_shape[2]/2))), 'symmetric')
            X = torch.FloatTensor(X)
            img_shape = np.transpose(np.array(list(np.shape(X))))
            output_size1 = math.floor((img_shape[1] - kernel_shape[1])/(stride)) + 1
            output_size2 = math.floor((img_shape[2] - kernel_shape[2])/(stride)) + 1
            output_shape = [channels,output_size1,output_size2]
            X_im2col = im2col2d(X,kernel,stride)
            
            weight = torch.reshape(kernel,(kernel_shape[0]*kernel_shape[1]*kernel_shape[2],1))
            # weight consists of only one weight vector. But the dimensionality of output space has to be 
            # num_filters. So we need to stack weight vectors horizontally and create num_filters number of 
            # feature maps
            for i in range(channels-1):
                weight2 = np.random.uniform(-0.1,0.1,size = (kernel_shape[0]*kernel_shape[1]*kernel_shape[2],1)) # Our input
                weight2 = torch.FloatTensor(weight2)
                weight2.requires_grad = True 
                weight = torch.cat((weight2, weight),1) # do this num_filters - 1 number of times
            conv_output = torch.t(X_im2col).mm(weight)
            conv_output = torch.reshape(conv_output,(output_shape))
            weight.retain_grad()
            return torch.nn.Parameter(conv_output), torch.nn.Parameter(weight),X_im2col

    def backprop_with_gradient_descent(self,y_pred,y,x):
        return

    def feedforward(self,x):
        
        return
    
    
# I havent converted the code below into a CNN_Engine class, because the code below is for experimentation 
# The CNN trains, but it would be better to make it library-esque so that it can be run like PyTorch
    
def im2col2d(X,kernel,stride,im_needed = True,shape_specified = False): # Assuming X and the kernel are 2D inputs
        # We transform the input matrix X into a matrix Ω(x) such that the columns correspond to the 
        # set of elements that will be multiplied by the kernel in each sliding convolution operation
        # Given x -> Ω(x), conv2d(x,kernel) = Ω(x)' * kernel where ' denotes a transpose operation

        X = X.detach().numpy()
        X = np.array(X)
        if not shape_specified: # If the kernel shape is not specified, then it is assumed that the argument kernel is a weight matrix
            kernel_shape = kernel.shape
        else: # If the kernel shape is specified, then it is assumed that the argument is the shape of the weight, not the weight itself
            kernel_shape = kernel

        img_shape = np.shape(X)
        a = np.empty((kernel_shape[0]*kernel_shape[1]*kernel_shape[2]))
        output_matrix = np.reshape(a,(kernel_shape[0]*kernel_shape[1]*kernel_shape[2],1))
        q = 0
        output_size1 = math.floor((img_shape[1] - kernel_shape[1])/(stride)) + 1
        output_size2 = math.floor((img_shape[2] - kernel_shape[2])/(stride)) + 1
        if im_needed:
            indicator_dict = {}            # Map (i,j,k) to (p,q)  (i,j,k): (p,q)
        coord_matrix = np.zeros((img_shape[0],img_shape[1],img_shape[2]),dtype='object')
        for k in range(0,img_shape[2]):
             for j in range(0,img_shape[1]):
                for i in range(0,img_shape[0]):
                    coord_matrix[i,j,k] = (i,j,k)
                    
        for k in range(0,img_shape[2],stride):
             for j in range(0,img_shape[1],stride):
                for i in range(0,img_shape[0],stride):
                        try:
                            vals = X[i:i+kernel_shape[0],j:j+kernel_shape[1],k:k+kernel_shape[2]]
                            vals = np.reshape(vals,(int(np.shape(vals)[2]*np.shape(vals)[1]*np.shape(vals)[0]),1))
                            output_matrix = np.hstack((output_matrix,vals))
                            indices = coord_matrix[i:i+kernel_shape[0],j:j+kernel_shape[1],k:k+kernel_shape[2]]
                            
                            indices = np.reshape(indices,(int(np.shape(indices)[2]*np.shape(indices)[1]*np.shape(indices)[0])))                            
                            coords = [(p,q) for p in range(len(indices))]
                            d = dict(zip(coords,list(indices)))
                            indicator_dict.update(d)
                            q+=1
                            
                        except:
                             continue
        
        if im_needed:
            reversed_indicator_dict = dict()
            for k,v in indicator_dict.items():
                reversed_indicator_dict.setdefault(v, []).append(k) # Reversed mapping now in order to 
        
        output_matrix = np.delete(output_matrix,0,1)
        output_matrix = torch.FloatTensor(output_matrix)
        if im_needed:
            return torch.nn.Parameter(output_matrix),reversed_indicator_dict
        else:
            return torch.nn.Parameter(output_matrix),-1
    
def conv2D(self,channels,X,stride,kernel_shape,padding = False,initialize_weights = True,*args): # filters = dimensionality of output space
            # If padding is enabled, we pad the input with zeros such that the input size 
            # remains the same if weights with stride 1 are applied to the input
            
            if initialize_weights:
                kernel = np.random.uniform(-0.1,0.1,size = (kernel_shape[0],kernel_shape[1],kernel_shape[2])) # Our input
             # Our input
                kernel = torch.FloatTensor(kernel)
                kernel.requires_grad = True
            else:
                kernel = args[0] # weights and bias must be given if initialise weights is disabled
                bias = args[1]
                kernel_shape = kernel.shape
            
            X = X.detach().numpy()
            if padding: # Can only pad during initialization -> weights and input shapes cannot change during feedforward and backpropagation
                if kernel_shape[1] % 2 == 0 and kernel_shape[2] % 2 == 0:
                    X = np.pad(X,((0,0),(math.floor(kernel_shape[1]/2)-1,math.floor(kernel_shape[1]/2)),(math.floor(kernel_shape[2]/2),math.floor(kernel_shape[2]/2)-1)), 'symmetric')
                elif kernel_shape[1] % 2 != 0 and kernel_shape[2] % 2 == 0:
                    X = np.pad(X,((0,0),(math.floor(kernel_shape[1]/2),math.floor(kernel_shape[1]/2)),(math.floor(kernel_shape[2]/2),math.floor(kernel_shape[2]/2)-1)), 'symmetric')
                elif kernel_shape[1] % 2 == 0 and kernel_shape[2] % 2 != 0:
                    X = np.pad(X,((0,0),(math.floor(kernel_shape[1]/2)-1,math.floor(kernel_shape[1]/2)),(math.floor(kernel_shape[2]/2),math.floor(kernel_shape[2]/2))), 'symmetric')
                else:
                    X = np.pad(X,((0,0),(math.floor(kernel_shape[1]/2),math.floor(kernel_shape[1]/2)),(math.floor(kernel_shape[2]/2),math.floor(kernel_shape[2]/2))), 'symmetric')
            
            X = torch.FloatTensor(X)
            
            img_shape = X.shape
            
            output_size1 = math.floor((img_shape[1] - kernel_shape[1])/(stride)) + 1
            output_size2 = math.floor((img_shape[2] - kernel_shape[2])/(stride)) + 1
            output_shape = [channels,output_size1,output_size2]
            
            X_im2col,im = im2col2d(X,kernel,stride)
            
            
            if initialize_weights:
                weight = torch.reshape(kernel,(kernel_shape[0]*kernel_shape[1]*kernel_shape[2],1))
                # weight consists of only one weight vector. But the dimensionality of output space has to be 
                # num_filters. So we need to stack weight vectors horizontally and create num_filters number of 
                # feature maps
                for i in range(channels-1):
                    weight2 = np.random.uniform(-0.1,0.1,size = (kernel_shape[0]*kernel_shape[1]*kernel_shape[2],1)) # Our input
                    weight2 = torch.FloatTensor(weight2)
                    weight2.requires_grad = True 
                    weight = torch.cat((weight2, weight),1) # do this num_filters - 1 number of times
                conv_output = torch.t(X_im2col).mm(weight)
                bias = torch.Tensor(np.random.uniform(-0.1,0.1,size = conv_output.shape))
                conv_output += bias
                conv_output = torch.reshape(conv_output,(output_shape))
                return torch.nn.Parameter(conv_output), torch.nn.Parameter(weight),X_im2col,im, output_shape,bias
            else:
                # Since weights are already initialised, the relevant channels are already dictated in the architecture. 
                # Therefore, conv output is just a matmul 
                conv_output = torch.t(X_im2col).mm(kernel) + bias
                return torch.nn.Parameter(conv_output),X_im2col
                
                
            

# For example, if you call out.backward() for some variable out that 
# involved x in its calculations then x.grad will hold ∂out/∂x      
# kernel = np.array([[[1,1],[1,1]]]) # format = channels * rows * columns
# X = np.array([[[1,2,3,1],[4,5,6,1],[7,8,9,1]]])
def maxpool(X,kernel_shape,stride,*args,return_architecture = False):
    # If the gradient of X (dz_dX) is specified, the output will be given along with dz_doutput
    
#     X_im2col,im = im2col2d(X,kernel_shape,stride,im_needed = True,shape_specified = True)
#     output = torch.zeros((X_im2col.shape[1],1))
#     output_size0 = math.floor((X.shape[0] - kernel_shape[0])/(stride)) + 1
#     output_size1 = math.floor((X.shape[1] - kernel_shape[1])/(stride)) + 1
#     output_size2 = math.floor((X.shape[2] - kernel_shape[2])/(stride)) + 1
#     output_shape = (output_size0,output_size1,output_size2)
#     for j in range(X_im2col.shape[1]):
#         output[j] = max(X_im2col[:,j])
    output_size1 = math.floor((X.shape[1] - kernel_shape[1])/(stride)) + 1
    output_size2 = math.floor((X.shape[2] - kernel_shape[2])/(stride)) + 1
    if return_architecture:
        im = {}
        imx = {} # inverted im
    for i in range(X.shape[0]):
        Xi = X[i,:,:]
        Xi = torch.reshape(Xi,(-1,Xi.shape[0],Xi.shape[1]))
        X_im2col,imi = im2col2d(Xi,kernel_shape,stride,im_needed = True,shape_specified = True)
        if i == 0:
            X_im2c = torch.zeros((X.shape[0],X_im2col.shape[0],X_im2col.shape[1]))
            output = torch.zeros((X.shape[0],X_im2col.shape[1]))
        X_im2c[i,:,:] = X_im2col
        if return_architecture:
            im[i] = imi
            imx[i] = {value: key for key in imi for value in imi[key]}
        for j in range(X_im2col.shape[1]):
            output[i,j] = max(X_im2col[:,j])
    output_shape = (X.shape[0],output_size1,output_size2)
    output = torch.reshape(output,output_shape)
#     if args and len(args) > 0:
#         dz_doutput = torch.zeros((X_im2col.shape[1],1))
#         dz_dX = args[0]
#         for j in range(X_im2col.shape[1]):
#             idx = torch.argmax(X_im2col[:,j])
#             dz_doutput[j] = dz_dX[idx,j]
#         dz_doutput = torch.reshape(dz_doutput,output_shape)
#         if initialise_weights == True:
#             activation = 'maxpool'
#             return output,dz_doutput,activation
#     else:
#         if initialise_weights == True:
#             activation = 'maxpool'
#             return output,activation 
#         else:

    if return_architecture:
        return output,output_shape,X_im2c,im,imx
    else:
        return output

def maxpool_im2col(X,kernel_shape,stride):
    output_size1 = math.floor((X.shape[1] - kernel_shape[1])/(stride)) + 1
    output_size2 = math.floor((X.shape[2] - kernel_shape[2])/(stride)) + 1
    
    im = {}
    
    for i in range(X.shape[0]):
        Xi = X[i,:,:]
        Xi = torch.reshape(Xi,(-1,Xi.shape[0],Xi.shape[1]))
        X_im2col,imi = im2col2d(Xi,kernel_shape,stride,im_needed = True,shape_specified = True)
        if i == 0:
            X_im2c = torch.zeros((X.shape[0],X_im2col.shape[0],X_im2col.shape[1]))
            output = torch.zeros((X.shape[0],X_im2col.shape[1]))
        X_im2c[i,:,:] = X_im2col
    
    return X_im2c
    
class MyReLU(torch.autograd.Function):
    """
    We can implement our own custom autograd Functions by subclassing
    torch.autograd.Function and implementing the forward and backward passes
    which operate on Tensors.
    """

    @staticmethod
    def forward(ctx, input):
        """
        In the forward pass we receive a Tensor containing the input and return
        a Tensor containing the output. ctx is a context object that can be used
        to stash information for backward computation. You can cache arbitrary
        objects for use in the backward pass using the ctx.save_for_backward method.
        """
        ctx.save_for_backward(input)
        return input.clamp(min=0)

    @staticmethod
    def backward(ctx, grad_output):
        """
        In the backward pass we receive a Tensor containing the gradient of the loss
        with respect to the output, and we need to compute the gradient of the loss
        with respect to the input.
        """
        input, = ctx.saved_tensors
        grad_input = grad_output.clone()
        grad_input[input < 0] = 0
        return grad_input
    
def softmax(y): # A bit lengthy for a simple softmax implementation but it was all I could do to avoid gradient related errors
#     y = y.squeeze()
    epsilon = 0.001
    y = y.detach().numpy()
    return torch.Tensor(np.exp(y)/(epsilon + sum(np.exp(y))))

def BatchNorm(X): # (X - mu) / sigma
    epsilon = 0.001  # To prevent overflow and ensure numerical stability
    bn = (X - torch.mean(X)) / (torch.std(X)+epsilon)   
    sigma.append(torch.std(X)+epsilon)
    return bn
    
def conv_bn_relu(X,channels,stride,kernel_shape,padding,activation = 'sigmoid',initialize_weights = True,batchnorm = True,*args): # Refactoring conv_batchnorm_relu as one layer
    if initialize_weights:
        output_1,weights,X_im2col,im,output_shape,bias = conv2D(0,channels,X,stride,kernel_shape,padding,initialize_weights = True,*args) # conv
    else:
        output1,X_im2col = conv2D(0,channels,X,stride,kernel_shape,padding,initialize_weights = False,*args) # conv
    if batchnorm == True:
        output_1 = BatchNorm(output_1) # batchnorm
    if activation == 'sigmoid':
        X_1 = sigmoid(output_1)
    elif activation == 'relu':
        X_1 = relu(output_1)
    else:
        X_1 = output_1
    if initialize_weights:
        return torch.nn.Parameter(X_1),torch.nn.Parameter(weights),torch.nn.Parameter(X_im2col),im,output_shape,bias,activation
    else:
        return output1,X_im2col

def flatten_and_dense(X,out_channels,*args,activation = 'sigmoid', initialise_weights = True): 
    """ Flattens the input and outputs a fully connected dense layer after applying an activation function
        Extra args correspond to weights and biases input"""
    
    shape = X.shape
    X = torch.reshape(X,(shape[0]*shape[1]*shape[2],1)) # Flatten
    if initialise_weights == True:
        weights = torch.Tensor(np.random.uniform(-0.1,0.1, size = (out_channels,len(X))))
        bias = torch.Tensor(np.random.uniform(-0.1,0.1, size = (out_channels,1)))
    else:
        weights = args[0]
        bias = args[1]
    if activation == 'sigmoid':
        output = sigmoid(weights.mm(X) + bias)
    else: # Only support sigmoid and relu for now
        output = relu(weights.mm(X) + bias)
    return output
                                                                     
def cross_entropy(y_pred,y):
        epsilon = 0.001 # To prevent overflow and ensure numerical stability
        return sum(-y*np.log(y_pred+epsilon))

def sigmoid(X):
    X = X.detach().numpy()
    X = torch.FloatTensor((1/(1+(np.exp(-X)))))
    return X

def getIndexes(indicator_matrix, val): 

    ''' Get desired coordinates of val in an indicator matrix. 
    val will be in the form of ijk cartesian coordinates'''    
    try:
        return indicator_matrix[val]
    except:
        return []

gradient_layerwise = {} # Map dz_dweights and dz_dX to each layer

relu = MyReLU.apply

padding = False # Mostly will never set this to true
sigma = []
num_examples = 2
# y = np.random.randint(0,2,size = (10,1))
y = np.zeros((10,num_examples))

for i in range(y.shape[1]):
    y[y_train[i],i] = 1
y_input = y   
y = Variable(torch.FloatTensor(y))
y.requires_grad = False

X_inp = x_train[0:num_examples,:,:]
X_inp = torch.FloatTensor(X_inp)

architecture = {}

#### Architecture: Shape it as you want ####
channels = 2
X_in = X_inp
X = X_inp[0:1,:,:]
y = torch.Tensor(y_input[:,0:1])

stride = 1
kernel_shape1 = np.array([1,7,7])
X_1,weights1,X_im2col,im1,output_shape1,bias1,activation1 = conv_bn_relu(X,channels,stride,kernel_shape1,padding,batchnorm = True,activation = "relu",initialize_weights = True)

architecture['layer1'] = np.array([X,X_1,weights1,bias1,X_im2col,im1,output_shape1,kernel_shape1,stride,'conv_bn_relu',None],dtype = 'object')
gradient_layerwise['layer1'] = np.array([torch.zeros(X_1.shape),torch.zeros(weights1.shape),torch.zeros(bias1.shape)],dtype='object')

# gradient layerwise format: layer i contains: dz_dXinput, dz_dXoutput, dz_dweights (Xoutput = Xinput*weights),dz_dbiasinput, dz_dbiasoutput
kernel_shape3 = (np.array([1,6,6]))
stride3 = 2
X_3,output_shape3,X_2_im2col,im3,imx = maxpool(X_1,kernel_shape3,stride3,return_architecture=True)


architecture['layer2'] = np.array([X_1,X_3,None,None,X_2_im2col,im3,output_shape3,kernel_shape3,stride3,'maxpool',imx])
gradient_layerwise['layer2'] = np.array([torch.zeros(X_3.shape),None,None],dtype='object')


stride = 3
kernel_shape2 = np.array([channels,5,5])

X_2,weights2,X_1_im2col,im2,output_shape2,bias2,activation2 = conv_bn_relu(X_3,channels,stride,kernel_shape2,padding,batchnorm = True,initialize_weights = True) # can change the number of channels to any integer value

architecture['layer3'] = np.array([X_1,X_2,weights2,bias2,X_1_im2col,im2,output_shape2,kernel_shape2,stride,'conv_bn_relu',None],dtype = 'object')
gradient_layerwise['layer3'] = np.array([torch.zeros(X_2.shape),torch.zeros(weights2.shape),torch.zeros(bias2.shape)],dtype='object')



print(X_3.shape)


# X3_output = torch.reshape(X_3,(-1,1))
# dz_weights2 = torch.zeros(weights2.shape)
# # print(X_1_im2col.shape)
# # 
# # print(X3_output.shape)   # (36,1)
# a = torch.t(X_1_im2col).mm(weights2)
# a = torch.reshape(a, X_2.shape)
# # print(a.shape)   # (1,11,11)
# # print(weights2.shape) # (16,1)
# dz_dX2 = torch.zeros(X_2.shape)
# dz_dX3 = gradient_layerwise['layer3'][0]
# dz_dX3 = torch.reshape(dz_dX3,X3_output.shape)
# imx = {value: key for key in im3 for value in im3[key]}

# for j in range(X_2_im2col.shape[1]):
#     idx = torch.argmax(X_2_im2col[:,j]).item()
# #             (p,q) = (idx,j) -> map this to ijk in X_2 using im3
#     value = imx[(idx,j)]
#     dz_dX2[value[0],value[1],value[2]] = dz_dX3[j]
# #     a[]
# dz_db2 = dz_dX2
# dz_dX2[X2 < 0 ] == 0
# dz_dX2 = sigmoid(dz_dX2)*(1-sigmoid(dz_dX2))
# dz_dX2 = torch.reshape(dz_dX2,(-1,1))
# print("Hi")
# print(dz_dX2.shape) # either relu' this or sigmoid' this

# print(X_1_im2col.shape)
# dz_dweights2 = dz_dX2*X_1_im2col

# dz_dX2 = torch.reshape(dz_dX2,output_shape3)

# X2 -> maxpool -> X3
# we have dz_dX3 -> need dz_dX2 
# we have dz_dweights4 -> need dz_weights2 

# relu(X3im2col * weights4) = X4
# X3 = maxpool(X2)
# maxpool(relu(X1im2col * weights2)) = X3o
# dz_dweights4 = given
# dz_X3 = given
# dz_dX2 = given 
# dz_dweights2 = dz_dX3o * dX3o_dweights2
# dX3_dweights2 = X1im2col in places where max was found


# dz_dX2 = dz_dX3 in places where max was found
# In other places it is 0


stride = 2
kernel_shape4 = np.array([channels,2,2])

channels = 10
X_4,weights4,X_3_im2col,im4,output_shape4,bias4,activation4 = conv_bn_relu(X_2,channels,stride,kernel_shape4,padding,batchnorm = True,initialize_weights = True)

architecture['layer4'] = np.array([X_3,X_4,weights4,bias4,X_3_im2col,im4,output_shape4,kernel_shape4,stride,'conv_bn_relu',None],dtype = 'object')
gradient_layerwise['layer4'] = np.array([torch.zeros(X_4.shape),torch.zeros(weights4.shape),torch.zeros(bias4.shape)],dtype='object')


# if layer is maxpool specify activation as maxpool 

# kernel_shape = (10,1,1)
# X_4,weights4,X_3_im2col,im4 = conv_bn_relu(X_3,channels,stride,kernel_shape,padding)

# kernel_shape = (10,2,2)
# X_5,weights5,X_4_im2col = conv_bn_relu(X_4,channels,stride,kernel_shape,padding)

# kernel_shape = (10,1,1) # no of channels in kernel must be the same as that in the input image, i.e. X_5 in this case
# Y,weights6, X_5_im2col = conv_bn_relu(X_5,channels,stride,kernel_shape,padding)
# Y = X_2
# X_2 = X_2.squeeze()

##### Experimental: Implementing Backpropagation 
y_pred = softmax(X_4)

architecture['layer5'] = np.array([X_4,y_pred,None,None,None,None,y_pred.shape,None,None,'softmax',None])
gradient_layerwise['layer5'] = np.array([np.zeros(y_pred.shape),None,None],dtype='object')

y_pred = torch.reshape(y_pred,y.size())
loss = Variable(cross_entropy(y_pred.squeeze(),y.squeeze()))
weights2.requires_grad = False
weights1.requires_grad = False
# weights3.requires_grad = False
weights4.requires_grad = False

dropout = 0.2
epochs = 250
bs = 10
arr = []
epsilon = 0.01

beta_1 = 0.9
beta_2 = 0.999

def adam(g,beta_1,beta_2,m,v,t):
    if not isinstance(g,np.ndarray):
        g = g.detach().numpy()
    if not isinstance(m,np.ndarray): 
        m = m.detach().numpy()
    if not isinstance(v,np.ndarray): 
        v = v.detach().numpy()
    m = beta_1 * m + (1 - beta_1) * g
    v = beta_2 * v + (1 - beta_2) * np.power(g, 2)
    m_hat = m / (1 - np.power(beta_1, t)) + (1 - beta_1) * g / (1 - np.power(beta_1, t))
    v_hat = v / (1 - np.power(beta_2, t))
    grad = lr * m_hat / (np.sqrt(v_hat) + epsilon)
    return torch.Tensor(grad),torch.Tensor(m),torch.Tensor(v)


for epoch in range(epochs): # training + backpropagation. Problem: too slow -> need to speed it up
    clip = 1000
    lr_multiplier = 0.2
    stride1 = 1
    stride2 = 2
    stride3 = 3

    gc.collect()
#     lr = 0.05
    lr = min(0.2,0.01*np.exp(lr_multiplier*epoch))
    
    
    
#     exp = 200
#     y = np.zeros((10,exp))
#     x_input = x_test[0:exp,:,:]
#     x_input = torch.FloatTensor(x_input)
#     val_loss = 0
#     num_correct = 0
#     for i in range(y.shape[1]):
#         y[y_test[i],i] = 1
#         y_val = torch.FloatTensor(y)
#     for index in range(exp):
#         X = BatchNorm(x_input[index,:,:])
#         y = np.array(torch.Tensor(y_val[:,index]))
#         X = torch.reshape(X,(1,X.shape[0],X.shape[1]))
#         # X = torch.reshape(y,(1,X.shape[0],X.shape[1]))
#         X_im2col,imX1 = im2col2d(X,kernel_shape1,stride1,im_needed = False,shape_specified = True)
#         X_1 = relu(BatchNorm(torch.t(X_im2col).mm(weights1) + bias1))
#         X_1 = torch.reshape(X_1,output_shape1)
#         X_1_im2col, imX2 = im2col2d(X_1,kernel_shape2,stride2,im_needed = False,shape_specified = True)
#         X_2 = relu(BatchNorm(torch.t(X_1_im2col).mm(weights2) + bias2))
#         X_2 = torch.reshape(X_2,output_shape2)
#         X_2_im2col, imX3 = im2col2d(X_2,kernel_shape3,stride3,im_needed = False,shape_specified = True)
#         X_3 = torch.t(X_2_im2col).mm(weights3) + bias3
#         X_x = X_3.squeeze()
#         y_pred = np.array(softmax(X_x))
#         val_loss += cross_entropy(y_pred.squeeze(),y.squeeze())
#         prediction = np.argmax(y_pred,axis=0)
#         actual = np.argmax(y,axis=0)
#     #     imshow(np.array(X.squeeze()))
#     #     print("Prediction is: {}".format(prediction))
#         if prediction == actual:
#             num_correct += 1
#     accuracy = num_correct / exp
#     accuracy *= 100
    
#     arr.append(loss)
#     loss = 0
    t = 1
    loss_new_method = 0

    # Initialising grad weights dictionary for each layer for the purpose of mini-batch gradient descent
    grad_weights = {}
    grad_bias = {}
    
    # Initialising first and second moments for each gradient and bias matrix in each layer
    m = {}
    v = {}
    
    for layer in range(len(architecture)):
        weightsi = architecture['layer{}'.format(layer+1)][2]
        biasi = architecture['layer{}'.format(layer+1)][3]
        if weightsi is not None and biasi is not None:
            grad_weights['layer{}'.format(layer+1)] = torch.zeros(weightsi.shape)
            grad_bias['layer{}'.format(layer+1)] = torch.zeros(biasi.shape)
            m['layer{}'.format(layer+1)] = np.array([torch.zeros(weightsi.shape),torch.zeros(biasi.shape)],dtype='object')
            v['layer{}'.format(layer+1)] = np.array([torch.zeros(weightsi.shape),torch.zeros(biasi.shape)],dtype='object')
        else:
            grad_weights['layer{}'.format(layer+1)] = None
            grad_bias['layer{}'.format(layer+1)] = None
            m['layer{}'.format(layer+1)] = [None,None]
            v['layer{}'.format(layer+1)] = [None,None]
    
    for j in range(num_examples):
        sigma = []  # Collecting the sigmas from the BatchNorm layer for the purposes of gradient descent
        # Y = (X - mu) /sigma
        # dY_dX = (1/sigma)*ones(X.shape)
        
        
        if j%bs == 0 or j==num_examples-1 and j != 0: 
            if j%10 == 0:
                print("Example: {}".format(j))
            for layer in range(len(architecture)):
                if not (grad_weights['layer{}'.format(layer+1)] is None) and grad_bias['layer{}'.format(layer+1)] is not None:
                    grad_weightsi = grad_weights['layer{}'.format(layer+1)]
                    grad_weightsi /= bs
                    grad_biasi = grad_bias['layer{}'.format(layer+1)] 
                    grad_biasi /= bs

                    # Adam optimisation

                    mw,mb = m['layer{}'.format(layer+1)]
                    vw,vb = v['layer{}'.format(layer+1)]
                    qw,mw,vw = adam(grad_weightsi,beta_1,beta_2,mw,vw,t) # Have obtained dw
                    qb,mb,vb = adam(grad_biasi,beta_1,beta_2,mb,vb,t) # Have obtained db 
                    
                    # Updating weights and biases now
                    try:
                        architecture['layer{}'.format(layer+1)][2] -= torch.Tensor(qw)
                    except:
                        architecture['layer{}'.format(layer+1)][2] -= torch.t(torch.Tensor(qw))
                    try:
                        architecture['layer{}'.format(layer+1)][3] -= torch.Tensor(qb)
                    except:
                        architecture['layer{}'.format(layer+1)][3] -= torch.t(torch.Tensor(qb))
                    
                    m['layer{}'.format(layer+1)][0] = torch.Tensor(mw)
                    m['layer{}'.format(layer+1)][1] = torch.Tensor(mb)
                    v['layer{}'.format(layer+1)][0] = torch.Tensor(vw)
                    v['layer{}'.format(layer+1)][1] = torch.Tensor(vb)

                    grad_weights['layer{}'.format(layer+1)] = torch.zeros(grad_weightsi.shape)
                    grad_bias['layer{}'.format(layer+1)] = torch.zeros(grad_biasi.shape)
            t+= 1
        
        
        X = X_in[j:j+1,:,:]
        y = torch.Tensor(y_input[:,j:j+1])
        architecture['layer1'.format(layer+1)][0] = X
        kernel_shape1 = architecture['layer1'.format(layer+1)][7]
        stride1 = architecture['layer1'.format(layer+1)][8]
        if kernel_shape1 is not None and not isinstance(kernel_shape1,int):
            X_input_1_im2col,imX = im2col2d(X,kernel_shape1,stride1,im_needed = False, shape_specified = True) 
            architecture['layer1'][4] = X_input_1_im2col
        else:
            architecture['layer1'][4] = None
        
        for layer in range(len(architecture)): # Feedforward from the first till the second last layer
            X_input,X_output,weightsi,biasi,X_input_1_im2col,imi,output_shapei,kernel_shapei,stridei,operationi,imx = architecture['layer{}'.format(layer+1)]
#             print("Operation is:{} and Layer is: {}".format(operationi,layer+1))
            # X_i_1_im2col is the im2col of the input
            # X_output is the convolution output after convolution is applied in this layer
            # 
            # Possible operations: will make architecture customizable so can keep appending
            # conv_bn_relu  (can be conv2d or conv3d)
            # conv_relu
            # conv_sigmoid
            # conv_bn_sigmoid
            # maxpool
            # softmax
            # flatten_dense_relu
            # flatten_dense_sigmoid
            # skip_connections (need output to retain the same shape as input for this so either do a 1x1 convolution or apply padding)
            
            if operationi == 'conv_bn_relu':
                conv_output = relu(BatchNorm(torch.t(X_input_1_im2col).mm(weightsi) + biasi))
                conv_output = torch.reshape(conv_output,output_shapei)
                architecture['layer{}'.format(layer+1)][1] = conv_output # resetting output as convolved shape
                if layer != len(architecture) - 1:
                    architecture['layer{}'.format(layer+2)][0] = conv_output # resetting intput of next layer as convolved shape
                    kernel_shapei__1 = architecture['layer{}'.format(layer+2)][7]
                    stridei__1 = architecture['layer{}'.format(layer+2)][8]
                    operationi__1 = architecture['layer{}'.format(layer+2)][9]
                    if kernel_shapei__1 is not None and not isinstance(kernel_shapei__1,int):
                        if operationi__1 == 'maxpool':
                            architecture['layer{}'.format(layer+2)][4] = maxpool_im2col(conv_output,kernel_shapei__1,stridei__1)
                        else:
                            architecture['layer{}'.format(layer+2)][4],imX = im2col2d(conv_output,kernel_shapei__1,stridei__1,im_needed = False, shape_specified = True)
                        # resetting input im2col of next layer as the im2col of the output of this layer
                    else:
                        architecture['layer{}'.format(layer+2)][4] = None
            elif operationi == 'conv_relu':
                conv_output = relu(torch.t(X_input_1_im2col).mm(weightsi) + biasi)
                conv_output = torch.reshape(conv_output,output_shapei)
                architecture['layer{}'.format(layer+1)][1] = conv_output # resetting output as convolved shape
                if layer != len(architecture) - 1:
                    architecture['layer{}'.format(layer+2)][0] = conv_output # resetting intput of next layer as convolved shape
                    kernel_shapei__1 = architecture['layer{}'.format(layer+2)][7]
                    stridei__1 = architecture['layer{}'.format(layer+2)][8]
                    operationi__1 = architecture['layer{}'.format(layer+2)][9]
                    if kernel_shapei__1 is not None and not isinstance(kernel_shapei__1,int):
                        if operationi__1 == 'maxpool':
                            architecture['layer{}'.format(layer+2)][4] = maxpool_im2col(conv_output,kernel_shapei__1,stridei__1)
                        else:
                            architecture['layer{}'.format(layer+2)][4],imX = im2col2d(conv_output,kernel_shapei__1,stridei__1,im_needed = False, shape_specified = True)
                        # resetting input im2col of next layer as the im2col of the output of this layer
                    else:
                        architecture['layer{}'.format(layer+2)][4] = None
            elif operationi == 'conv_bn_sigmoid':
                conv_output = sigmoid(BatchNorm(torch.t(X_input_1_im2col).mm(weightsi) + biasi))
                conv_output = torch.reshape(conv_output,output_shapei)
                architecture['layer{}'.format(layer+1)][1] = conv_output # resetting output as convolved shape
                if layer != len(architecture) - 1:
                    architecture['layer{}'.format(layer+2)][0] = conv_output # resetting intput of next layer as convolved shape
                    kernel_shapei__1 = architecture['layer{}'.format(layer+2)][7]
                    stridei__1 = architecture['layer{}'.format(layer+2)][8]
                    operationi__1 = architecture['layer{}'.format(layer+2)][9]
                    if kernel_shapei__1 is not None and not isinstance(kernel_shapei__1,int):
                        if operationi__1 == 'maxpool':
                            architecture['layer{}'.format(layer+2)][4] = maxpool_im2col(conv_output,kernel_shapei__1,stridei__1)
                        else:
                            architecture['layer{}'.format(layer+2)][4],imX = im2col2d(conv_output,kernel_shapei__1,stridei__1,im_needed = False, shape_specified = True)
                        # resetting input im2col of next layer as the im2col of the output of this layer
                    else:
                        architecture['layer{}'.format(layer+2)][4] = None
            elif operationi == 'conv_sigmoid':   
                conv_output = sigmoid(torch.t(X_input_1_im2col).mm(weightsi) + biasi)
                conv_output = torch.reshape(conv_output,output_shapei)
                architecture['layer{}'.format(layer+1)][1] = conv_output # resetting output as convolved shape
                if layer != len(architecture) - 1:
                    architecture['layer{}'.format(layer+2)][0] = conv_output # resetting intput of next layer as convolved shape
                    kernel_shapei__1 = architecture['layer{}'.format(layer+2)][7]
                    stridei__1 = architecture['layer{}'.format(layer+2)][8]
                    operationi__1 = architecture['layer{}'.format(layer+2)][9]
                    if kernel_shapei__1 is not None and not isinstance(kernel_shapei__1,int):
                        if operationi__1 == 'maxpool':
                            architecture['layer{}'.format(layer+2)][4] = maxpool_im2col(conv_output,kernel_shapei__1,stridei__1)
                        else:
                            architecture['layer{}'.format(layer+2)][4],imX = im2col2d(conv_output,kernel_shapei__1,stridei__1,im_needed = False, shape_specified = True)
                        # resetting input im2col of next layer as the im2col of the output of this layer
                    else:
                        architecture['layer{}'.format(layer+2)][4] = None
            elif operationi == 'maxpool':
                maxpool_output = maxpool(X_input,kernel_shapei,stridei)
                
                maxpool_output = torch.reshape(maxpool_output,output_shapei)
                
                if layer != len(architecture) - 1:
                    architecture['layer{}'.format(layer+2)][0] = maxpool_output
                    kernel_shapei__1 = architecture['layer{}'.format(layer+2)][7]
                    stridei__1 = architecture['layer{}'.format(layer+2)][8]
                    if kernel_shapei__1 is not None and not isinstance(kernel_shapei__1,int):
#                         a,imx = im2col2d(maxpool_output,kernel_shapei__1,stridei__1,im_needed = False, shape_specified = True)
#                         print(a.shape)
                        architecture['layer{}'.format(layer+2)][4],imX = im2col2d(maxpool_output,kernel_shapei__1,stridei__1,im_needed = False, shape_specified = True)
                        # potential use for a convolution: good idea to store all im2col representations of input
                    else:
                        architecture['layer{}'.format(layer+2)][4] = None
            elif operationi == 'flatten_dense_relu':
                # kernel_shapei in this case refers to the output channels: stride for dense layer will be None
                output = flatten_and_dense(X_input,kernel_shapei,weightsi,biasi,activation = 'relu',initialise_weights = False)
                architecture['layer{}'.format(layer+1)][1] = output
            elif operationi == 'flatten_dense_sigmoid':
                # kernel_shapei in this case refers to the output channels: stride for dense layer will be None
                output = flatten_and_dense(X_input,kernel_shapei,weightsi,biasi,activation = 'sigmoid',initialise_weights = False)
                architecture['layer{}'.format(layer+1)][1] = output
            elif operationi == 'softmax':
                Xin = architecture['layer{}'.format(layer+1)][0]
                output = softmax(Xin).squeeze()
                architecture['layer{}'.format(layer+1)][1] = output  
            if layer == len(architecture) - 1:
                y_pred = architecture['layer{}'.format(len(architecture))][1]
                loss_new_method += cross_entropy(y_pred.squeeze(),y.squeeze())
                
        # Backpropagation from the last layer till the first layer
        X = X_in[j:j+1,:,:]
        y = torch.Tensor(y_input[:,j:j+1])
        
        for layer in range(len(architecture)-1,-1,-1): 
            
            X_input,X_output,weightsi,biasi,X_input_im2col,imi,output_shapei,kernel_shapei,stridei,operationi,imxi = architecture['layer{}'.format(layer+1)]      
#             print("Operation is:{} and Layer is: {}".format(operationi,layer+1))
            if operationi == 'softmax': # Last layer -> Dont apply softmax in any layer other than the last layer!
                # not taking gradients here because we need dz_dX(secondlastlayer) which is y_pred - y
                continue
            
            if operationi == 'conv_bn_relu' or operationi == 'conv_relu' or operationi == 'conv_sigmoid' or operationi == 'conv_bn_sigmoid': 
                operationi__1 = architecture['layer{}'.format(layer+2)][9]  
                if operationi__1 == 'softmax':
                    y_pred = architecture['layer{}'.format(layer+2)][1]  
                    y_pred = torch.reshape(y_pred,y.shape)
                    dz_dXi = y_pred - y
                    dz_dXi[dz_dXi > clip] = 0  # Gradient Clipping
                    dz_dXi[dz_dXi < -clip] = 0 # Gradient Clipping
                    
                    if operationi == 'conv_sigmoid' or operationi == 'conv_bn_sigmoid':
                        X_output = torch.reshape(X_output,dz_dXi.shape)
                        dz_dXi *= sigmoid(X_output)*(1-sigmoid(X_output))    # Taking the derivative of the sigmoid function
                    else:
                        dz_dXi[X_output.squeeze() <= 0] = 0
                        
                    gradient_layerwise['layer{}'.format(layer+1)][0] = dz_dXi
                    dz_dbi = torch.reshape(dz_dXi,biasi.shape)
                    gradient_layerwise['layer{}'.format(layer+1)][2] = dz_dbi
                    try:
                        dz_dweightsi = (dz_dXi).mm(torch.t(X_input_im2col))    # dz_dweights3 = dz_dX3 * dX3_dweights3  (chain rule)
                    except:
                        dz_dweightsi = (dz_dXi).mm(X_input_im2col)
                    dz_dweightsi[dz_dweightsi > clip] = 0  # Gradient Clipping
                    dz_dweightsi[dz_dweightsi < -clip] = 0
                    gradient_layerwise['layer{}'.format(layer+1)][1] = dz_dweightsi
                elif operationi__1 == 'maxpool':
#                     print(gradient_layerwise['layer{}'.format(layer+2)][0].shape) # dz_dx3
#                     print(gradient_layerwise['layer{}'.format(layer+1)][0].shape) # dz_dx2
#                     print(gradient_layerwise['layer{}'.format(layer)][0].shape)   # dz_dx1
                    continue 
                else:
                    weightsi__1 = architecture['layer{}'.format(layer+2)][2] # weights2
                    dz_dXi__1 = gradient_layerwise['layer{}'.format(layer+2)][0] # dz_dX2 -> backpropagated from maxpool
                    output_shapei__1 = architecture['layer{}'.format(layer+2)][6]
                    if len(dz_dXi__1.shape) == 3:
                        dz_dXi__1 = torch.reshape(dz_dXi__1,(-1,output_shapei__1[0]))
                    imi__1 = architecture['layer{}'.format(layer+2)][5]  # im3
                    try:
                        Y = weightsi__1.mm(dz_dXi__1)
                    except:
                        Y = weightsi__1.mm(torch.t(dz_dXi__1)) # One or the other will work :)
                    dz_dXi = torch.zeros(X_output.shape)  # X_1
                    output_shape_current_layer = architecture['layer{}'.format(layer+1)][6]
                    bias_current_layer = architecture['layer{}'.format(layer+1)][3]  # bias2
                    X_im2col_prev_layer = architecture['layer{}'.format(layer+1)][4] # X_im2col
                    for i in range(np.shape(X_output)[0]):
                        for j in range(np.shape(X_output)[1]):
                            for k in range(np.shape(X_output)[2]):
                                idxs = getIndexes(imi__1,(i,j,k))
                                dz_dXi[i,j,k] = sum([Y[i[0],i[1]] for i in idxs])

                    dz_dXi[dz_dXi > clip] = 0  # Gradient Clipping
                    dz_dXi[dz_dXi < -clip] = 0 # Gradient Clipping
                    if operationi == 'conv_sigmoid' or operationi == 'conv_bn_sigmoid':
                        X_output = torch.reshape(X_output,dz_dXi.shape)
                        dz_dXi *= sigmoid(X_output)*(1-sigmoid(X_output))    # Taking the derivative of the sigmoid function
                    else:
                        dz_dXi[X_output <= 0] = 0    # Weirdly the network seems to be performing better if i dont take the derivative of the activation function

                    dz_dXi = torch.reshape(dz_dXi,(output_shape_current_layer[1]*output_shape_current_layer[2],-1))
                    dz_dbi = torch.reshape(dz_dXi,bias_current_layer.shape)

                    dz_dweightsi = X_im2col_prev_layer.mm(dz_dXi)    
                    dz_dweightsi[dz_dweightsi > clip] = 0  # Gradient Clipping
                    dz_dweightsi[dz_dweightsi < -clip] = 0 # Gradient Clipping
                    gradient_layerwise['layer{}'.format(layer+1)][0] = torch.Tensor(dz_dXi)
                    gradient_layerwise['layer{}'.format(layer+1)][1] = torch.Tensor(dz_dweightsi)
                    gradient_layerwise['layer{}'.format(layer+1)][2] = torch.Tensor(dz_dbi)
                    
            if operationi == 'maxpool':
                imx = architecture['layer{}'.format(layer+1)][10]
                
                dz_dXinput = torch.zeros((X_input.shape))
                dz_dXoutput = gradient_layerwise['layer{}'.format(layer+1)][0]
#                 X_input_im2col = torch.reshape(X_input_im2col,(X_input_im2col.shape[0],X_input_im2col.shape[2],X_input_im2col.shape[1]))
                
                dz_dXoutput = torch.reshape(dz_dXoutput,(output_shapei[0],X_input_im2col.shape[2])) # dz_dX2 here
                
                for i in range(output_shapei[0]):
                    for j in range(X_input_im2col.shape[2]):
                        Xi2ci = X_input_im2col[i,:,:]
                        idx = torch.argmax(Xi2ci[:,j]).item()
                        value = imx[i][(idx,j)]
                        dz_dXinput[value[0],value[1],value[2]] += float(dz_dXoutput[i,j])
                    
#                 dz_dXinput = torch.reshape(dz_dXinput,output_shapei)
                dz_dXinput[X_input <= 0] = 0
                gradient_layerwise['layer{}'.format(layer)][0] = torch.Tensor(dz_dXinput) # dz_X2 =
    
            if gradient_layerwise['layer{}'.format(layer+1)][1] is not None:
                try:
                    grad_weights['layer{}'.format(layer+1)] += gradient_layerwise['layer{}'.format(layer+1)][1]
                except:
                    grad_weights['layer{}'.format(layer+1)] += torch.t(gradient_layerwise['layer{}'.format(layer+1)][1])
            if gradient_layerwise['layer{}'.format(layer+1)][2] is not None:
                try:
                    grad_bias['layer{}'.format(layer+1)] += gradient_layerwise['layer{}'.format(layer+1)][2]
                except:
                    grad_bias['layer{}'.format(layer+1)] += torch.t(gradient_layerwise['layer{}'.format(layer+1)][2])
        
        
        
#         # Feedforward
# #         X = BatchNorm(X_input[j:j+1,:,:])
# #         y = torch.Tensor(y_input[:,j:j+1])
#         X = BatchNorm(X_in[j:j+1,:,:])
# #         print(X_input.shape)
#         y = torch.Tensor(y_input[:,j:j+1])
#         X_im2col,imX1 = im2col2d(X,kernel_shape1,stride1,im_needed = False,shape_specified = True)
#         X_1 = relu(BatchNorm(torch.t(X_im2col).mm(weights1) + bias1))
#         X_1 = torch.reshape(X_1,output_shape1)
# #]         X_1m = maxpool(X_1,)
    
#         X_1_im2col, imX2 = im2col2d(X_1,kernel_shape2,stride2,im_needed = False,shape_specified = True)
#         X_2 = relu(BatchNorm(torch.t(X_1_im2col).mm(weights2) + bias2))
#         X_2 = torch.reshape(X_2,output_shape2)
#         X_2_im2col, imX3 = im2col2d(X_2,kernel_shape3,stride3,im_needed = False,shape_specified = True)
#         X_3 = torch.t(X_2_im2col).mm(weights3) + bias3
#         X_x = X_3.squeeze()
#         y_pred = softmax(X_x)
        
#         y_pred = torch.reshape(y_pred,y.size())

#         loss += Variable(cross_entropy(y_pred.squeeze(),y.squeeze()),requires_grad=True)

#         ## BACKPROPAGATION ##
        
#         dz_dX3 = y_pred - y     # Gradient of loss function wrt last layer output before softmax
        
#         dz_dX3[dz_dX3 > clip] = 0 # Gradient clipping
#         dz_dX3[dz_dX3 < -clip] = 0 # Gradient clipping
# #         dz_dX3[torch.t(X_3) <= 0.000001] = 0     # For ReLu

# #         dz_dX3 = sigmoid(dz_dX3)*(1-sigmoid(dz_dX3))  # for sigmoid

#         dz_db3 = torch.t(dz_dX3)
        
#         # ' = derivative
#         # X3 = sigmoid(bn(X2im2col*weights3))
#         # dX3_dX2im2col = sigmoid'(weights3)
#         dz_dweights3 = (dz_dX3).mm(torch.t(X_2_im2col))    # dz_dweights3 = dz_dX3 * dX3_dweights3  (chain rule)
#         dz_dweights3[dz_dweights3 > clip] = 0  # Gradient Clipping
#         dz_dweights3[dz_dweights3 < -clip] = 0 # Gradient Clipping
# #         dz_dweights3 *= sigma[2]
#         gc.collect()

#         Y = weights3.mm(dz_dX3) # Y is the same shape as X_2im2col -> dz_dX2 (im2col version) = dz_dX3 * dX3_dX2im2col = Y as dX3_dX2im2col = weights3
#         # might need to do sigmoid'(weights3) or weights3()

#         # dz_dX | (i,j,k) = sum (Y) at all locations (p,q) in X_2im2col where X_2(i,j,k) is placed. Makes sense coz Y is the im2col version of dz_dX2
 
#         dz_dX2 = torch.zeros(X_2.size())
        
#         for i in range(np.shape(X_2)[0]):
#             for j in range(np.shape(X_2)[1]):
#                 for k in range(np.shape(X_2)[2]):
#                     idxs = getIndexes(im3,(i,j,k))
#                     dz_dX2[i,j,k] = sum([Y[i[0],i[1]] for i in idxs])
# #         dz_dX2[X_2 <= 0.000001] = 0
#         dz_dX2 = torch.reshape(dz_dX2,(np.shape(X_2)[1]*np.shape(X_2)[2],-1))
# #         dz_dX2[dz_dX2 < 0] = 0        # For ReLu
        
#         dz_dX2[dz_dX2 > clip] = 0  # Gradient Clipping
#         dz_dX2[dz_dX2 < -clip] = 0 # Gradient Clipping
# #         dz_dX2 = sigmoid(dz_dX2)*(1-sigmoid(dz_dX2))    #for sigmoid

#         dz_db2 = dz_dX2
        
#         # ' = derivative
#         # X2 = sigmoid(bn(X1im2col*weights2))
#         # dX2_dX1im2col = X1im2col*sigmoid'(weights2)

#         dz_dweights2 = X_1_im2col.mm(dz_dX2)
#         dz_dweights2[dz_dweights2 > clip] = 0  # Gradient Clipping
#         dz_dweights2[dz_dweights2 < -clip] = 0 # Gradient Clipping
# #         print(dz_dweights2[dz_dweights2 != 0])
# #         dz_dweights2 *= sigma[1]
# #         print(dz_dX2.shape)
        
#         Y = weights2.mm(torch.t(dz_dX2))

#         dz_dX1 = torch.zeros(X_1.size())

#         for i in range(np.shape(X_1)[0]):
#             for j in range(np.shape(X_1)[1]):
#                 for k in range(np.shape(X_1)[2]): 
#                     idxs = getIndexes(im2,(i,j,k))

#                     dz_dX1[i,j,k] = sum([Y[p[0],p[1]] for p in idxs])

# #         dz_dX1[X_1 <= 0.000001] = 0
#         dz_dX1 = torch.reshape(dz_dX1,(np.shape(X_1)[1]*np.shape(X_1)[2],-1))

#         gc.collect()
        
# #         dz_dX1[dz_dX1 < 0] = 0 # For Relu
        
#         dz_dX1[dz_dX1 > clip] = 0 # Gradient clipping
#         dz_dX1[dz_dX1 < -clip] = 0 # Gradient clipping
        
# #         dz_dX1 = sigmoid(dz_dX1)*(1-sigmoid(dz_dX1)) # For sigmoid

#         dz_db1 = dz_dX1
    
        
#         dz_dweights1 = X_im2col.mm(dz_dX1)
#         dz_dweights1[dz_dweights1 > clip] = 0  # Gradient Clipping
#         dz_dweights1[dz_dweights1 < -clip] = 0 # Gradient Clipping
# #         dz_dweights1 *= sigma[0]
        
#         grad_weights3 += torch.t(dz_dweights3)
#         grad_weights2 += dz_dweights2
#         grad_weights1 += dz_dweights1
#         grad_bias3 += dz_db3
#         grad_bias2 += dz_db2
#         grad_bias1 += dz_db1
        gc.collect()
        
    print("Epoch: {}".format(epoch))
    print("Training Loss: {}".format(loss_new_method/num_examples)) 
#     print("Validation Accuracy: {} %".format(accuracy))
#     print("Validation Loss: {} %".format(val_loss/exp))
    
#         grad_weights3 = 0
#         grad_weights2 = 0
#         grad_weights1 = 0
#         grad_bias3 = 0
#         grad_bias2 = 0
#         grad_bias1 = 0

#### Architecture: Shape it as you want ####

# Architecture
# conv batchnorm relu (stride 2, 10 feature maps) - > 16
# conv batchnorm relu (stride 2, 10 feature maps) - > 8
# conv batchnorm relu (stride 2, 10 feature maps) - > 4
# conv batchnorm relu (stride 2, 10 feature maps) - > 2
# conv batchnorm relu (stride 2, 10 feature maps) - > 1
# softmax
# y.squeeze()

# We essentially only want the gradient of the loss with respect to the weights, so we can perform grad descent.
# We don't need anything else!


torch.Size([2, 9, 9])
Example: 0
Epoch: 0
Training Loss: 2.399521827697754
Example: 0
Epoch: 1
Training Loss: 2.125241279602051
Example: 0
Epoch: 2
Training Loss: 1.9875072240829468
Example: 0
Epoch: 3
Training Loss: 1.866539716720581
Example: 0
Epoch: 4
Training Loss: 1.8326191902160645
Example: 0
Epoch: 5
Training Loss: 1.8230822086334229
Example: 0
Epoch: 6
Training Loss: 1.8207684755325317
Example: 0
Epoch: 7
Training Loss: 1.8228144645690918
Example: 0
Epoch: 8
Training Loss: 1.825018048286438
Example: 0
Epoch: 9
Training Loss: 1.826524019241333
Example: 0
Epoch: 10
Training Loss: 1.8275446891784668
Example: 0
Epoch: 11
Training Loss: 1.8282361030578613
Example: 0
Epoch: 12
Training Loss: 1.8287067413330078
Example: 0
Epoch: 13
Training Loss: 1.8290292024612427
Example: 0
Epoch: 14
Training Loss: 1.8292518854141235
Example: 0
Epoch: 15
Training Loss: 1.8294060230255127
Example: 0
Epoch: 16
Training Loss: 1.8294786214828491
Example: 0
Epoch: 17
Training Loss: 1.8295328617095947
Exa

KeyboardInterrupt: 

In [1]:
import torch
x = torch.tensor([1.5], requires_grad=True)
a = torch.nn.Parameter(torch.tensor([2.]))
b = torch.nn.Parameter(torch.tensor([10.]))
y = x*a
z = y+0.5*b
temp = z.backward()
print('gradients of a: %0.2f and b: %0.2f' % (a.grad.item(), b.grad.item()))

gradients of a: 1.50 and b: 0.50


In [2]:
num_correct = 0
num_examples = 200
rand = 1000
import numpy as np
import tensorflow as tf
import torch
from torch.autograd import Variable
import cv2
import IPython
y = np.zeros((10,num_examples))
val_loss = 0
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
def imshow(img):
    _,ret = cv2.imencode('.jpg', img)
    i = IPython.display.Image(data=ret)
    IPython.display.display(i)
x_input = x_test[rand:num_examples+rand,:,:]
x_input = torch.FloatTensor(x_input)
for i in range(y.shape[1]):
    y[y_test[i+rand],i] = 1
    y_val = Variable(torch.FloatTensor(y))
for index in range(num_examples):
    print(index)
    X = x_input[index,:,:]
    y = np.array(torch.Tensor(y_val[:,index]))
    X = torch.reshape(X,(1,X.shape[0],X.shape[1]))
    # X = torch.reshape(y,(1,X.shape[0],X.shape[1]))
    X_im2col,imX1 = im2col2d(X,kernel_shape1,stride1,im_needed = False,shape_specified = True)
    X_1 = relu(BatchNorm(torch.t(X_im2col).mm(weights1) + bias1))
    X_1 = torch.reshape(X_1,output_shape1)
    X_1_im2col, imX2 = im2col2d(X_1,kernel_shape2,stride2,im_needed = False,shape_specified = True)
    X_2 = relu(BatchNorm(torch.t(X_1_im2col).mm(weights2) + bias2))
    X_2 = torch.reshape(X_2,output_shape2)
    X_2_im2col, imX3 = im2col2d(X_2,kernel_shape3,stride3,im_needed = False,shape_specified = True)
    X_3 = BatchNorm(torch.t(X_2_im2col).mm(weights3) + bias3)
    X_x = X_3.squeeze()
    y_pred = np.array(softmax(X_x))
    val_loss += cross_entropy(y_pred.squeeze(),y.squeeze())
    prediction = np.argmax(y_pred,axis=0)
    actual = np.argmax(y,axis=0)
    print("Prediction is: {}".format(prediction))
    if prediction == actual:
        num_correct += 1
    if prediction == 5:
        imshow(np.array(X.squeeze()))
accuracy = num_correct / num_examples
accuracy *= 100
print(accuracy)
print(val_loss/num_examples)

0


NameError: name 'im2col2d' is not defined

In [ ]:
# y_pred = softmax(X_2)

# z = cross_entropy(y,y_pred) = sum(-y*log(y_pred))

# dz/X_2 = softmax(X_2) - y

# X_2 = X_im2col2'*weights2                       X_2 is not softmaxed

# dX_2/dweights2 = X_im2col2'

# X_im2col2 = relu(bn(X_im2col1'*weights1)

# X_1 = col2im(X_im2col2

# dz/weights2 = dz/dX_2*dX_2/dweights2

# dz/weights1 = dz/dX_2*dX_2/dX_im2col2*dX_im2col2/dweights1

# dX_2/X_im2col2 = weights2'

# dz/weights1 = dz/dX_2*weights2'*relu(X_im2col1')

# X_2'*X1_im2col' = dW

# dX_col = 

In [30]:
weights1, bias1 = weights['layer1']
weights2, bias2 = weights['layer2']
weights3, bias3 = weights['layer3']

In [16]:
print(accuracy)

72.39999999999999


In [15]:
import pickle
with open('Vanilla_CNN_MNIST_weights_80%val','wb') as f:
    pickle.dump(weights_bias,f)

In [9]:
a = X_1_im2col
a = a.detach().numpy()
a = np.array(a)
print(a.size)

2500


In [7]:
print(y_pred,y)

NameError: name 'y_pred' is not defined

In [14]:
weights_bias = {'layer1': [weights1,bias1],'layer2':[weights2,bias2],'layer3':[weights3,bias3]}

In [1]:
print(im2)

NameError: name 'im2' is not defined

In [21]:
print(y_pred)

tensor([[0.0768],
        [0.0679],
        [0.0547],
        [0.0733],
        [0.1591],
        [0.1272],
        [0.0700],
        [0.1795],
        [0.0756],
        [0.1158]])


In [56]:
import sys
np.set_printoptions(threshold=sys.maxsize)
torch.set_printoptions(threshold=5000)

In [120]:
print(gradient_layerwise['layer4'][0].shape)

torch.Size([10, 1])


In [44]:
print(architecture)

{'layer1': array([tensor([[[  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
            0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
            0.,   0.,   0.,   0.,   0.,   0.],
         [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
            0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
            0.,   0.,   0.,   0.,   0.,   0.],
         [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
            0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
            0.,   0.,   0.,   0.,   0.,   0.],
         [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
            0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
            0.,   0.,   0.,   0.,   0.,   0.],
         [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
            0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
            0.,   0.,   0.,   0., 

In [ ]:
# Story idea: All search engines stop working -> Cant access any information online